In [2]:
import sys
import pandas as pd
from sqlalchemy import create_engine
from urllib.parse import quote_plus

# ── CONFIG ──────────────────────────────────────────────────────────
user     = 'etl_user1'
password = quote_plus('3December2000@')
host     = '127.0.0.1'
port     = '5432'
db       = 'etl_demo'
engine   = create_engine(
    f'postgresql+psycopg2://{user}:{password}@{host}:{port}/{db}'
)

def run_checks():
    # 1) Raw table checks
    df_raw = pd.read_sql("SELECT * FROM public.sales_raw", engine,
                         parse_dates=['order_date','ship_date'])
    assert df_raw['order_id'].notnull().all(), "Null order_id found"
    assert df_raw['order_id'].is_unique,      "Duplicate order_id found"
    assert (df_raw['sales'] >= 0).all(),      "Negative sales detected"
    assert df_raw['discount'].between(0,1).all(), "Invalid discount (>1 or <0)"
    print("✅ sales_raw quality checks passed")

    # 2) Summary table checks
    df_sum = pd.read_sql("SELECT * FROM public.sales_summary", engine)
    assert not df_sum.empty,                 "sales_summary is empty"
    print("✅ sales_summary quality checks passed")

if __name__ == "__main__":
    try:
        run_checks()
    except AssertionError as e:
        print(f"❌ Data quality check failed: {e}")
        sys.exit(1)
    print("🎉 All data quality checks passed")


✅ sales_raw quality checks passed
✅ sales_summary quality checks passed
🎉 All data quality checks passed
